In [1]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "spark-3.5.3-bin-hadoop3"

In [2]:
import findspark
findspark.init()

In [3]:
import multiprocessing
print(multiprocessing.cpu_count())

8


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSV Loader") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext  # Get the existing SparkContext

25/03/20 14:01:13 WARN Utils: Your hostname, Duncans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.18.95.63 instead (on interface en0)
25/03/20 14:01:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 14:01:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/20 14:01:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
reddit_rdd = sc.textFile("reddit_test/reddit_cleaned.csv")
header=reddit_rdd.first()
reddit_rdd=reddit_rdd.filter(lambda row : row!= header)

In [65]:
%%time
pos_words_en=set()
neg_words_en=set()
pos_words_es=set()
neg_words_es=set()
pos_words_fr=set()
neg_words_fr=set()
pos_words_tr=set()
neg_words_tr=set()
pos_words_it=set()
neg_words_it=set()

with open('sentiment_txts/positive_words_en.txt','r') as file:
    for line in file:
        pos_words_en.add(line.strip())
with open('sentiment_txts/negative_words_en.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_en.add(line.strip())
with open('sentiment_txts/positive_words_es.txt','r') as file:
    for line in file:
        pos_words_es.add(line.strip())
with open('sentiment_txts/negative_words_es.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_es.add(line.strip())
with open('sentiment_txts/positive_words_fr.txt','r') as file:
    for line in file:
        pos_words_fr.add(line.strip())
with open('sentiment_txts/negative_words_fr.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_fr.add(line.strip())
with open('sentiment_txts/positive_words_tr.txt','r') as file:
    for line in file:
        pos_words_tr.add(line.strip())
with open('sentiment_txts/negative_words_tr.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_tr.add(line.strip())
with open('sentiment_txts/positive_words_it.txt','r') as file:
    for line in file:
        pos_words_it.add(line.strip())
with open('sentiment_txts/negative_words_it.txt','r',encoding='latin-1') as file:
    for line in file:
        neg_words_it.add(line.strip())

CPU times: user 12.6 ms, sys: 4.01 ms, total: 16.6 ms
Wall time: 15.8 ms


In [50]:
reddit_rdd.take(5)

['hi1vsag,When you scheduled your booster with CVS does it just give you the option of Vaccines: COVID-19 (Vaccine brand) or does it specifically say booster',
 'hi1vs7i,"Didn\'t stop prices there though. New Zealand and Canada grew at about the same rate through COVID.   I agree that non-resident ownership should be stopped, but it\'s also probably not going to change anything."',
 'hi1vs5n,I’m just waiting until after pregnancy to get vaccinated.  I have a special needs daughter due to having fever during pregnancy and I’m not willing to risk the vaccine fever right now. I have strong antibodies from Covid that I had before I got pregnant. (I keep taking blood tests to check) There’s a difference between anti vaccine and deciding to wait based on different factors',
 'hi1vs5v,"*The first duty of every Starfleet officer is to the truth. Whether it\'s scientific truth, or historical truth, or personal truth. It is the guiding principle upon which Starfleet is based. If you can\'t find 

In [51]:
import re
#input: line of rdd, output: list of (id,word) pairs for each cleaned word in text
def reddit_id_word_pair(line):
    id, text = line.split(",",1) 
    cleaned_text=re.sub(r'[^a-zA-Z\s]','',text.strip()).lower()
    return [(id,word) for word in re.split(r'\s+',cleaned_text)]

In [52]:
def reddit_word_to_sentiment(pair):
    id, word = pair
    score=0
    if word in pos_words_en:
        score+= 1
    if word in neg_words_en:
        score-=1
    return id, score

In [53]:
reddit_id_word_pair_rdd=reddit_rdd.flatMap(reddit_id_word_pair)

In [54]:
reddit_sentiment_pair_rdd=reddit_id_word_pair_rdd.map(reddit_word_to_sentiment)

In [55]:
reddit_comment_scores=reddit_sentiment_pair_rdd.reduceByKey(lambda x,y: x+y)

In [56]:
reddit_positive_scores=reddit_comment_scores.filter(lambda x: x[1]>0)
reddit_neutral_scores=reddit_comment_scores.filter(lambda x: x[1]==0)
reddit_negative_scores=reddit_comment_scores.filter(lambda x: x[1]<0)

In [62]:
total=reddit_comment_scores.count()
print(total)
reddit_positive_percent=reddit_positive_scores.count()/total
reddit_neutral_percent=reddit_neutral_scores.count()/total
reddit_negative_percent=reddit_negative_scores.count()/total
print(f"pos: {reddit_positive_percent} neutral: {reddit_neutral_percent} neg: {reddit_negative_percent}")

100
pos: 0.34 neutral: 0.26 neg: 0.4


In [69]:
def twitter_id_word_pair(line):
    id, lang, text = line.split(",",2) 
    cleaned_text=re.sub(r'[^a-zA-Z\s]','',text.strip()).lower()
    return [(id,lang,word) for word in re.split(r'\s+',cleaned_text)]

def twitter_word_to_sentiment(pair):
    id, lang, word = pair
    score=0
    if lang == "en" or lang == 'und':
        if word in pos_words_en:
            score+= 1
        if word in neg_words_en:
            score-=1
        return id, score
    elif lang == "es":
        if word in pos_words_es:
            score+= 1
        if word in neg_words_es:
            score-=1
        return id, score
    elif lang == "fr":
        if word in pos_words_fr:
            score+= 1
        if word in neg_words_fr:
            score-=1
        return id, score
    elif lang == "tr":
        if word in pos_words_tr:
            score+= 1
        if word in neg_words_tr:
            score-=1
        return id, score
    else: #italian
        if word in pos_words_it:
            score+= 1
        if word in neg_words_it:
            score-=1
        return id, score
    return id, score

In [66]:
twitter_rdd = sc.textFile("twitter_test/twitter_cleaned.csv")
header=twitter_rdd.first()
twitter_rdd=twitter_rdd.filter(lambda row : row!= header)

In [67]:
twitter_rdd.take(5)

['1244051646071611394,es,"Ante cualquier enfermedad respiratoria, no te automediques. #PrevenciónCoronavirus #Coronavirus #COVID19 https://t.co/IgB2izYi1p"',
 '1244051645039706112,es,"#ATENCIÓN En el Terminal Nuevo Circo se implementan medidas como el uso de la mascarilla por parte de usuari@s y conductores que hacen vida en dichas instalaciones, parte de los esfuerzos unificados para impedir la propagación del #Coronavirus #28Marzo @NicolasMaduro  @ErikaPSUV https://t.co/yD8g0EAcQz"',
 '1244051645975191557,en,"“People are just storing up. They are staying at home freezing things. I talk to dealers around the state and out of the area, they are all out of freezers,” said owner Frank Mertz.  https://t.co/gsqghMqJs9  @KQDS_FOX21 #minneapolis #mn #covid19 #coronavirus"',
 '1244051646750928897,es,"Si empezaste a trabajar, necesitas dar de alta a tus beneficiarios ante el #IMSS, ahora lo puedes hacer desde tu domicilio a través de internet y evita filas | #SanaDistancia #QuédateEnCasa  #Cor

In [70]:
twitter_id_word_pair_rdd=twitter_rdd.flatMap(twitter_id_word_pair)
twitter_sentiment_pair_rdd=twitter_id_word_pair_rdd.map(twitter_word_to_sentiment)
twitter_comment_scores=twitter_sentiment_pair_rdd.reduceByKey(lambda x,y: x+y)

In [71]:
twitter_positive_scores=twitter_comment_scores.filter(lambda x: x[1]>0)
twitter_neutral_scores=twitter_comment_scores.filter(lambda x: x[1]==0)
twitter_negative_scores=twitter_comment_scores.filter(lambda x: x[1]<0)

In [72]:
total=twitter_comment_scores.count()
print(total)
twitter_positive_percent=twitter_positive_scores.count()/total
twitter_neutral_percent=twitter_neutral_scores.count()/total
twitter_negative_percent=twitter_negative_scores.count()/total
print(f"pos: {twitter_positive_percent} neutral: {twitter_neutral_percent} neg: {twitter_negative_percent}")

100
pos: 0.17 neutral: 0.28 neg: 0.55
